In [511]:
import os
import json
from pathlib import Path
from datetime import datetime

import pandas as pd
import numpy as np
from tqdm import tqdm

from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import torch

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler


In [512]:
def json_to_dataframe() -> pd.DataFrame:
        """
            Converts the JSON data into a pandas dataframe.

            :arg:
                None
            :return:
                weather_data (pandas dataframe): dataframe containing the weather data
            """

        folder_path = "/Users/jaya/Downloads/Git/Github/weather-prediction/artifacts/data_ingestion/data"
        file_names = os.listdir(folder_path)
        
        json_filepaths = [os.path.join(folder_path, name) for name in file_names if os.path.isfile(os.path.join(folder_path, name))]
        weather_data = None
        for filepath in json_filepaths:
            json_file = json.load(Path(filepath).open())
            for day in json_file['SiteRep']['DV']['Location']['Period']:
                df = pd.DataFrame(day['Rep'])
                df['date'] = day['value']
                weather_data = pd.concat([weather_data, df], ignore_index=True)
        return weather_data


In [513]:
data_headers = {
"D": "Wind Direction(compass)",
"G": "Wind Gust(mph)",
"H": "Screen Relative Humidity(%)",
"P": "Pressure(hpa)",
"S": "Wind Speed(mph)",
"T": "Temperature(C)",
"V": "Visibility(m)",
"W": "Weather Type",
"Pt": "Pressure Tendency",
"Dp": "Dew Point(C)",
"$": "Minutes Since 12o Clock"
}

compass_directions_map = {
"N": 1,
"NNE": 2,
"NE": 3,
"ENE": 4,
"E": 5,
"ESE": 6,
"SE": 7,
"SSE": 8,
"S": 9,
"SSW": 10,
"SW": 11,
"WSW": 12,
"W": 13,
"WNW": 14,
"NW": 15,
"NNW": 16 }

Pressure_tendency_map = {"F": 0, "R": 1, "S": 2}

column_datatypes = {
"Wind Direction(compass)": int,
"Wind Gust(mph)": float,
"Screen Relative Humidity(%)": float,
"Pressure(hpa)": float,
"Wind Speed(mph)": float,
"Temperature(C)": float,
"Visibility(m)": float,
"Weather Type": int,
"Pressure Tendency": int,
"Dew Point(C)": float,
"Minutes Since 12o Clock": int,
"date": "datetime64[ns]"
}

def clean_dataframe(weather_data: pd.DataFrame):
    weather_data.rename(columns=data_headers, inplace=True)
    weather_data.dropna(how='any', inplace=True)
    weather_data['Wind Direction(compass)'] = weather_data['Wind Direction(compass)'].map(compass_directions_map)
    weather_data['Pressure Tendency'] = weather_data['Pressure Tendency'].map(Pressure_tendency_map)
    weather_data['date'] = weather_data['date'].str.replace('Z','')
    weather_data = weather_data.astype(column_datatypes)
    weather_data['date'] = pd.to_datetime(weather_data['date'], format='%Y-%m-%d')
    return weather_data

def validate_data_columns(weather_data: pd.DataFrame) -> bool:
    try:
        validation_status = None
        data = weather_data
        all_columns = list(data.columns)
        schema_columns = list(column_datatypes.keys())
        if not sorted(all_columns) == sorted(schema_columns):
            validation_status = False
        else:
            validation_status = True
            for column, datatype in column_datatypes.items():
                if not data[column].dtype == datatype:
                    validation_status = False
        return validation_status
    except Exception as e:
        raise e


In [514]:
target_variables = ['Wind Direction(compass)', 'Pressure(hpa)', 'Wind Speed(mph)', 'Temperature(C)', 'Visibility(m)', 'Weather Type']
target_column_names = ['Wind Direction(compass) (t+1)', 'Pressure(hpa) (t+1)', 'Wind Speed(mph) (t+1)', 'Temperature(C) (t+1)', 'Visibility(m) (t+1)', 'Weather Type (t+1)']

def transform_date_feature(weather_data) -> pd.DataFrame:
    weather_data['day'] = weather_data['date'].dt.day
    weather_data['month'] = weather_data['date'].dt.month
    weather_data['year'] = weather_data['date'].dt.year
    weather_data.drop('date', axis=1, inplace=True)
    return weather_data

def make_target_features(weather_data) -> pd.DataFrame:
    weather_data.sort_values(by=['year', 'month', 'day', 'Minutes Since 12o Clock'], kind='mergesort', inplace=True)
    target = weather_data[target_variables].shift(-1)
    target.columns = target_column_names
    target = pd.concat([weather_data, target], axis=1)
    target.dropna(inplace=True)
    return target

# def split_data(self, x, y):
#     X_train, y_train, X_test, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
#     return X_train, y_train, X_test, y_test

# def min_max_scaler(x:pd.DataFrame) -> (pd.DataFrame, MinMaxScaler):
#     scaler = MinMaxScaler()

#     scaled_x = scaler.fit_transform(x)
#     return pd.DataFrame(scaled_x, columns=x.columns), scaler

# def invert_min_max_scaler(x:pd.DataFrame, scaler: MinMaxScaler) -> pd.DataFrame:
#     original_data = scaler.inverse_transform(x)

#     return pd.DataFrame(original_data, x.columns)


In [515]:
parameters = {
  "input_dim": 14,
  "output_dim": 1,
  "hidden_dim": 20,
  "learning_rate": 0.001,
  "batch_size": 100,
  "num_batches": 62,
  "num_epochs": 100
}

class WeatherModel(nn.Module):
    def __init__(self, parameters:dict):
        input_dim=parameters["input_dim"]
        output_dim=parameters["output_dim"]
        hidden_dim=parameters["hidden_dim"]

        super(WeatherModel, self).__init__()

        # Shared hidden layers
        self.shared_layer = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU()
        )

        # Output layers for each output
        self.temperature_layer = nn.Linear(hidden_dim, output_dim)
        self.visibility_layer = nn.Linear(hidden_dim, output_dim)
        self.wind_direction_layer = nn.Linear(hidden_dim, output_dim)
        self.wind_speed_layer = nn.Linear(hidden_dim, output_dim)
        self.weather_type_layer = nn.Linear(hidden_dim, output_dim)
        self.pressure_layer = nn.Linear(hidden_dim, output_dim)

    def forward(self, input):
        # Pass through shared hidden layer
        shared_output = self.shared_layer(input)

        # Separate output predictions
        temperature = self.temperature_layer(shared_output)
        visibility = self.visibility_layer(shared_output)
        wind_direction = self.wind_direction_layer(shared_output)
        wind_speed = self.wind_speed_layer(shared_output)
        weather_type = self.weather_type_layer(shared_output)
        pressure = self.pressure_layer(shared_output)

        return wind_direction, pressure, wind_speed, temperature, visibility, weather_type


In [516]:
class WeatherDataLoader(Dataset):
    def __init__(self, data:pd.DataFrame, features:list, target:list):
        self.X = torch.tensor(data[features].values,
                              dtype=torch.float32)
        self.y = torch.tensor(data[target].values, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


In [517]:
feature_columns_names = ['Wind Direction(compass)', 'Wind Gust(mph)', 'Screen Relative Humidity(%)', 'Pressure(hpa)', 'Wind Speed(mph)', 'Temperature(C)', 'Visibility(m)', 'Weather Type', 'Pressure Tendency', 'Dew Point(C)', 'Minutes Since 12o Clock', 'day', 'month', 'year']
target_column_names = ['Wind Direction(compass) (t+1)', 'Pressure(hpa) (t+1)', 'Wind Speed(mph) (t+1)', 'Temperature(C) (t+1)', 'Visibility(m) (t+1)', 'Weather Type (t+1)']

# Data normalization
def normalize_data(df, feature_columns, target_columns):
    scaler = StandardScaler()
    df[feature_columns] = scaler.fit_transform(df[feature_columns])
    df[target_columns] = scaler.fit_transform(df[target_columns])
    return df

def train(weather_data:pd.DataFrame):

    weather_data = normalize_data(weather_data, feature_columns_names, target_column_names)
    
    model = WeatherModel(parameters)
    weather_data_class = WeatherDataLoader(data=weather_data, features=feature_columns_names, target=target_column_names)
    dataloader = DataLoader(weather_data_class, batch_size=parameters["batch_size"], shuffle=True)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=parameters["learning_rate"])

    for epoch in range(parameters["num_epochs"]):
        all_preds = []
        all_labels = []
        running_loss = 0.0

        with tqdm(dataloader, unit="batch") as tepoch:
            for idx, (X, y) in enumerate(tepoch):
                tepoch.set_description(f"Epoch {epoch + 1}")

                # Zero the gradients
                optimizer.zero_grad()
                # Forward pass
                wind_direction, pressure, wind_speed, temperature, visibility, weather_type = model(X)

                # Compute loss
                loss_wind_direction = criterion(wind_direction, y[:, 0].view(-1, 1))
                loss_pressure = criterion(pressure, y[:, 1].view(-1, 1))
                loss_wind_speed = criterion(wind_speed, y[:, 2].view(-1, 1))
                loss_temperature = criterion(temperature, y[:, 3].view(-1, 1))
                loss_visibility = criterion(visibility, y[:, 4].view(-1, 1))
                loss_weather_type = criterion(weather_type, y[:, 5].view(-1, 1))
                # total loss
                loss = (loss_wind_direction + loss_pressure + loss_wind_speed + loss_temperature + loss_visibility + loss_weather_type)

                # Backpropagation and optimization
                loss.backward()
                optimizer.step()

                running_loss += loss.item()
                batch_preds = torch.cat([
                    wind_direction.unsqueeze(1), pressure.unsqueeze(1), wind_speed.unsqueeze(1), 
                    temperature.unsqueeze(1), visibility.unsqueeze(1), weather_type.unsqueeze(1)
                ], dim=1)
                all_preds.append(batch_preds.detach().cpu().numpy())
                all_labels.append(y.detach().cpu().numpy())

                tepoch.set_postfix(loss=running_loss / (idx + 1))

            all_preds = np.concatenate(all_preds, axis=0)
            all_labels = np.concatenate(all_labels, axis=0)

            all_preds = all_preds.reshape(-1, len(target_column_names))
            all_labels = all_labels.reshape(-1, len(target_column_names))

            mse = mean_squared_error(all_labels, all_preds)
            rmse = np.sqrt(mse)
            r2 = r2_score(all_labels, all_preds)
            mae = mean_absolute_error(all_labels, all_preds)
            
            # Print metrics for the epoch
            print(f'Epoch {epoch + 1}/{parameters["num_epochs"]}, Loss: {running_loss / len(dataloader):.4f}')
            print(f'MSE: {mse:.4f}, RMSE: {rmse:.4f}, R²: {r2:.4f}, MAE: {mae:.4f}')


In [518]:
print("Process Starting")
print("Parsing Data: Working..")
weather_data = json_to_dataframe()
print("Parsing Data: Success")
print("Cleaning Data: Working..")
weather_data = clean_dataframe(weather_data)
print("Cleaning Data: Success")
print("Validating Data: Working..")
validation_status = validate_data_columns(weather_data)

if validation_status:
    print("Validating Data: Success")
    print("Transforming Data: Working..")
    weather_data = transform_date_feature(weather_data)
    print("Transforming Data: Success")
    print("Target Variable Prep: Working..")
    weather_data = make_target_features(weather_data)
    print("Target Variable Prep: Success")
    print("Preparing for Model Training: Working..")
    train(weather_data)
    print("Model Training: Success")

else:
    print("Validating Data: Failed")




Process Starting
Parsing Data: Working..
Parsing Data: Success
Cleaning Data: Working..
Cleaning Data: Success
Validating Data: Working..
Validating Data: Success
Transforming Data: Working..
Transforming Data: Success
Target Variable Prep: Working..
Target Variable Prep: Success
Preparing for Model Training: Working..


Epoch 1: 100%|██████████| 75/75 [00:00<00:00, 373.79batch/s, loss=5.56]


Epoch 1/100, Loss: 5.5612
MSE: 0.9277, RMSE: 0.9632, R²: 0.0723, MAE: 0.7635


Epoch 2: 100%|██████████| 75/75 [00:00<00:00, 382.64batch/s, loss=3.55]


Epoch 2/100, Loss: 3.5466
MSE: 0.5917, RMSE: 0.7692, R²: 0.4083, MAE: 0.5961


...

Epoch 98/100, Loss: 1.2723
MSE: 0.2120, RMSE: 0.4604, R²: 0.7880, MAE: 0.2696


Epoch 99: 100%|██████████| 75/75 [00:00<00:00, 360.49batch/s, loss=1.27]


Epoch 99/100, Loss: 1.2736
MSE: 0.2121, RMSE: 0.4606, R²: 0.7879, MAE: 0.2696


Epoch 100: 100%|██████████| 75/75 [00:00<00:00, 363.47batch/s, loss=1.27]


Epoch 100/100, Loss: 1.2709
MSE: 0.2117, RMSE: 0.4602, R²: 0.7883, MAE: 0.2693
Model Training: Success


In [520]:
#torch.save(model.state_dict(), "/Users/jaya/Downloads/Git/Github/weather-prediction/artifacts/model_trainer/models/model.pth")
